THIS SECTION IS PREP AND PRETESTS

In [1]:
#This works for CPU
#!pip install -U transformers
#!pip install -U accelerate
#!pip install -U datasets
#!pip install -U bertviz
#!pip install -U umap-learn
#!pip install seaborn --upgrade
#!pip install evaluate
#!pip install -U openpyxl

In [2]:
#Extras when using GPU
#!pip install nvidia_smi
#!pip install pip install pynvml
#import os
#os.environ["TOKENIZERS_PARALLELISM"] = "false"
#the pip install over nvidia_smi gave this warning. the above line is to follow these instructions
    #huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
    #To disable this warning, you can either:
    #	- Avoid using `tokenizers` before the fork if possible
    #	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)

In [3]:
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA version: {torch.version.cuda}")
print(f"Is CUDA available: {torch.cuda.is_available()}")
print(f"CUDA device name: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'No GPU detected'}")


PyTorch version: 2.5.1+cu124
CUDA version: 12.4
Is CUDA available: True
CUDA device name: NVIDIA RTX A2000 Laptop GPU


In [4]:
import subprocess
import time
import torch
import nvidia_smi
monitor_script_path = "/home/charlie/Desktop/Practical/GPU_Measuring/GPU_Performance2.py"

monitor_process = subprocess.Popen(['python', monitor_script_path])
print(f"GPU Monitoring has started: {time.strftime('%H:%M:%S')}")

#Larger matrix means more gpu utilisation
matrix_size=3000
Loop_Amount=5000
a = torch.randn(matrix_size, matrix_size, device='cuda')
for _ in range(Loop_Amount):  
    b = torch.matmul(a, a)

monitor_process.terminate() 
print(f"GPU Monitoring has ended: {time.strftime('%H:%M:%S')}")

GPU Monitoring has started: 16:42:48
GPU Monitoring has ended: 16:44:17


THE ACTUAL PROCESS STARTS FROM THE CELL BELOW ONWARDS 

In [5]:
#Alternatively if it can load all dataframes without crashing
import pandas as pd

input_file = '/home/charlie/Desktop/Data_Subset/Masterfile/subset_1.csv'
chunk_size = 10000

try:
    df = pd.concat(pd.read_csv(input_file, chunksize=chunk_size))
    dropped_columns=["local_orig", "local_resp", "tunnel_parents", "detailed-label"]
    df.drop(columns=dropped_columns, inplace=True)
    df['label']=df['label'].replace({"Benign":0,"Malicious":1})#Converting all Benign to 0 and Malcious to 1
    print(df.shape)
    print(df['label'].value_counts())
    print(df[:2])
    print("Successfully loaded all dataframes")
except Exception as e:
    print(f"CRITICAL ERROR: {e}")

print("Cell Completed")

(12800, 19)
label
0    6400
1    6400
Name: count, dtype: int64
             ts                 uid      id.orig_h  id.orig_p  \
0  1.536233e+09  CMidSA4riIrgA8iWfj  156.249.40.90          3   
1  1.536233e+09   C03NhMRUsZDAoHhyg    12.89.9.154          3   

         id.resp_h  id.resp_p proto service  duration  orig_bytes  resp_bytes  \
0  192.168.100.111         10  icmp     NaN  0.000005        80.0         0.0   
1  192.168.100.111         13  icmp     NaN  0.000005        56.0         0.0   

  conn_state  missed_bytes history  orig_pkts  orig_ip_bytes  resp_pkts  \
0        OTH             0     NaN          2            136          0   
1        OTH             0     NaN          2            112          0   

   resp_ip_bytes  label  
0              0      0  
1              0      0  
Successfully loaded all dataframes
Cell Completed


/tmp/ipykernel_18779/1903815677.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['label']=df['label'].replace({"Benign":0,"Malicious":1})#Converting all Benign to 0 and Malcious to 1


In [6]:
#splitting data
import sklearn
from sklearn.model_selection import train_test_split
#below specified 0.3 of df is seperated to test, with a distrubtion of label column
train, test = train_test_split(df,test_size=0.3,stratify=df['label'])
#creates validation set by taking 1/3 of test
test, validation= train_test_split(test, test_size=1/3,stratify=test['label'])

print("(rows,columns)\n")
print(f"Data Frame:{df.shape}")
print(f"Training data:{train.shape}")
print(f"Test data:{test.shape}")
print(f"validation data:{validation.shape}")
print("Cell Completed")

(rows,columns)

Data Frame:(12800, 19)
Training data:(8960, 19)
Test data:(2560, 19)
validation data:(1280, 19)
Cell Completed


In [7]:
#convert dataframes in dataset dictionary objects, uses chunking to work with large datasets
from datasets import Dataset, DatasetDict, load_dataset, concatenate_datasets

chunk_size = 10000
try:
    def df_chunking(df, chunk_size=chunk_size):
        for start in range(0, len(df),chunk_size):
            yield Dataset.from_pandas(df.iloc[start:start + chunk_size],preserve_index=True)

    #make each dataframe split a chunk 
    train_chunks= list(df_chunking(train))
    test_chunks= list(df_chunking(test))
    validation_chunks= list(df_chunking(validation))

    #Concatenates each dataframe chunk into datasets
    train_dataset = concatenate_datasets(train_chunks)
    test_dataset = concatenate_datasets(test_chunks)
    validation_dataset = concatenate_datasets(validation_chunks)



    #makes data dictionaries
    dataset= DatasetDict(
        { "train": train_dataset,
          "test": test_dataset,
          "validation": validation_dataset
        }
    )

except Exception as e:
    print(f"CRITICAL ERROR: {e}")

print(f"dataset:{dataset}")
print("Cell Completed")

/home/charlie/anaconda3/envs/Current_Env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


dataset:DatasetDict({
    train: Dataset({
        features: ['ts', 'uid', 'id.orig_h', 'id.orig_p', 'id.resp_h', 'id.resp_p', 'proto', 'service', 'duration', 'orig_bytes', 'resp_bytes', 'conn_state', 'missed_bytes', 'history', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'label', '__index_level_0__'],
        num_rows: 8960
    })
    test: Dataset({
        features: ['ts', 'uid', 'id.orig_h', 'id.orig_p', 'id.resp_h', 'id.resp_p', 'proto', 'service', 'duration', 'orig_bytes', 'resp_bytes', 'conn_state', 'missed_bytes', 'history', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'label', '__index_level_0__'],
        num_rows: 2560
    })
    validation: Dataset({
        features: ['ts', 'uid', 'id.orig_h', 'id.orig_p', 'id.resp_h', 'id.resp_p', 'proto', 'service', 'duration', 'orig_bytes', 'resp_bytes', 'conn_state', 'missed_bytes', 'history', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'label', '__index_level_0__'],
        num_rows: 1280


In [8]:
#Get info on distilberts tokeniser
from transformers import AutoTokenizer
model_checkpoint="distilbert-base-uncased"
distilbert_tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
distilbert_tokenizer

DistilBertTokenizerFast(name_or_path='distilbert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [25]:
# Convert any strings to numeric representations
import torch

# Define the tokenization function
try: 
    def tokenize(batch):
        # Construct the log entry as a string
        log_entry = (
            f"{batch['ts']} {batch['uid']} {batch['id.orig_h']}:{batch['id.orig_p']} -> "
            f"{batch['id.resp_h']}:{batch['id.resp_p']} {batch['proto']} {batch['service']} "
            f"duration: {batch['duration']} orig_bytes: {batch['orig_bytes']} resp_bytes: {batch['resp_bytes']} "
            f"conn_state: {batch['conn_state']}"
        )
        # Apply the Hugging Face tokenizer
        return distilbert_tokenizer(log_entry, padding="max_length", truncation=True, max_length=512)

except Exception as e:
    print(f"CRITICAL ERROR: {e}")

# Extract the first two rows (returns a dictionary of lists)
sample = dataset['train'][:2]

# Convert the first sample into a dictionary (for testing)
first_sample = {key: sample[key][0] for key in sample}  # Extract the first log entry as a dict

# Test the tokenization on a single sample
print(f"Tokenized sample:\n{tokenize(first_sample)}\n")

# Apply tokenization to the entire dataset
# batched=True ensures that the function is applied to batches of rows
encoded_dataset = dataset.map(tokenize, batched=True, batch_size=512, drop_last_batch=True)
print("Cell Completed")


Tokenized sample:
{'input_ids': [101, 16710, 2575, 19317, 19841, 23499, 1012, 4720, 28756, 2620, 2475, 1039, 2620, 11475, 2072, 2475, 2595, 2140, 2475, 2860, 2361, 2480, 2860, 22540, 3654, 17613, 1012, 16923, 1012, 2531, 1012, 11118, 1024, 10700, 2549, 1011, 1028, 19975, 1012, 8574, 1012, 13092, 1012, 10550, 1024, 4720, 20842, 2683, 22975, 2361, 3904, 9367, 1024, 1016, 2063, 1011, 5757, 2030, 8004, 1035, 27507, 1024, 1014, 1012, 1014, 24501, 2361, 1035, 27507, 1024, 1014, 1012, 1014, 9530, 2078, 1035, 2110, 1024, 1055, 2692, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Map: 100%|██████████| 1024/1024 [00:00<00:00, 3858.14 examples/s]

Cell Completed


In [26]:
print(tokenize(dataset['train'][0]))  # Run on the first sample


{'input_ids': [101, 16710, 2575, 19317, 19841, 23499, 1012, 4720, 28756, 2620, 2475, 1039, 2620, 11475, 2072, 2475, 2595, 2140, 2475, 2860, 2361, 2480, 2860, 22540, 3654, 17613, 1012, 16923, 1012, 2531, 1012, 11118, 1024, 10700, 2549, 1011, 1028, 19975, 1012, 8574, 1012, 13092, 1012, 10550, 1024, 4720, 20842, 2683, 22975, 2361, 3904, 9367, 1024, 1016, 2063, 1011, 5757, 2030, 8004, 1035, 27507, 1024, 1014, 1012, 1014, 24501, 2361, 1035, 27507, 1024, 1014, 1012, 1014, 9530, 2078, 1035, 2110, 1024, 1055, 2692, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [27]:
from torch.utils.data import DataLoader
try:


    for i in range(5):
        print(f"Sample {i} length: {len(encoded_dataset['train'][i]['input_ids'])}")

        
except Exception as e:
    print(f"CRITICAL ERROR: {e}")
print("Cell Completed")

CRITICAL ERROR: object of type 'int' has no len()
Cell Completed


In [28]:
from torch.utils.data import DataLoader
try:


    dataloader = DataLoader(encoded_dataset['train'], batch_size=8)
    for batch in dataloader:
        print(f"Sample batch shape: {batch['input_ids'].shape}")
        break  # Check the
        
except Exception as e:
    print(f"CRITICAL ERROR: {e}")
print("Cell Completed")

CRITICAL ERROR: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'NoneType'>
Cell Completed


In [29]:
# Compare dropped batch and see if the balance is lost
df = encoded_dataset['train'].to_pandas()

# Print shape and label distribution
print(f"Dataset shape: {df.shape}")
print("Label distribution:")
print(df['label'].value_counts()) 
print("Successfully loaded all dataframes")


Dataset shape: (8704, 22)
Label distribution:
label
0    4360
1    4344
Name: count, dtype: int64
Successfully loaded all dataframes


In [30]:
#Loads model and passess config for classifier
from transformers import AutoModelForSequenceClassification, AutoConfig
import torch

label2id={"Benign":0,"Malicious":1}
id2label={0:"Benign",1:"Malicious"}
model_checkpoint="distilbert-base-uncased"


num_labels=len(label2id)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
config = AutoConfig.from_pretrained(model_checkpoint,label2id=id2label,id2label=label2id)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, config=config).to(device)

print(f"Labels: {model.config.id2label}")
print(f"Model: {model_checkpoint}")
print(f"Computer: {device}")
print("Model Config /n")
print(f": {config}")
print("Cell Completed")


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Labels: {'Benign': 0, 'Malicious': 1}
Model: distilbert-base-uncased
Computer: cuda
Model Config /n
: DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "Benign": 0,
    "Malicious": 1
  },
  "initializer_range": 0.02,
  "label2id": {
    "0": "Benign",
    "1": "Malicious"
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.48.0",
  "vocab_size": 30522
}

Cell Completed


In [31]:
# Build compute metrics function
import evaluate
import numpy as np

accuracy = evaluate.load("accuracy")

def compute_metrics_evaluate(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)
print("Cell Completed")

Cell Completed


In [32]:
#Training arguement 
from transformers import TrainingArguments

batch_size = 32
training_dir = "train_dir"

training_args = TrainingArguments(
                                  output_dir=training_dir,
                                  overwrite_output_dir = True,
                                  num_train_epochs = 2,
                                  learning_rate = 2e-5,
                                  per_device_train_batch_size = batch_size,
                                  per_device_eval_batch_size = batch_size,
                                  weight_decay = 0.01,
                                  evaluation_strategy = 'epoch',

)

print(f"Training Arugment: {training_args}")
print("Cell Completed")

Training Arugment: TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=None,
eval_strategy=IntervalSt

/home/charlie/anaconda3/envs/Current_Env/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [33]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    compute_metrics=compute_metrics_evaluate,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['validation'],
    tokenizer=distilbert_tokenizer
)

/tmp/ipykernel_18779/3748774873.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [34]:
try:
    trainer.train()
except Exception as e:
    print(f"CRITICAL ERROR: {e}")

CRITICAL ERROR: The size of tensor a (8) must match the size of tensor b (512) at non-singleton dimension 1
